In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.models import load_model

In [3]:
# Loading ANN train model, scalar pickle and one hot encoding pickle file 
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)

with open('label_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)
    


In [10]:
# Example of input data

input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Female',
    'Age' : 42,
    'Tenure' : 4,
    'Balance' : 20000,
    'NumOfProducts' : 3,
    'HasCrCard' : 1,
    'IsActiveMember': 1,
    'EstimatedSalary' : 35000
}

In [11]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Ansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_data_df = pd.DataFrame([input_data])

In [13]:
input_data_df['Gender'] = label_encoder_gender.fit_transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,42,4,20000,3,1,1,35000


In [14]:
input_data = pd.concat([input_data_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,42,4,20000,3,1,1,35000,1.0,0.0,0.0


In [16]:
scaled_input = scalar.transform(input_data)
scaled_input

array([[-0.53598516, -1.09499335,  0.29493847, -0.3483691 , -0.89825077,
         2.53355998,  0.64920267,  0.97481699, -1.13763226,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
# Predicting the values
prediction  = model.predict(scaled_input)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step


array([[0.61951655]], dtype=float32)

In [22]:
result = prediction[0][0]
result 

0.61951655

In [23]:
print("Likely to churn" if result>0.5 else "Not likely to churn")

Likely to churn
